# Deep Analysis: Phase Transition for Pool Splitting

Explores adversarial splitting strategies:

1. Equal split (baseline)
2. Sybil split (keep all pledge in one pool, zero-pledge the rest)
3. Optimal split (numerically optimize pledge distribution)
4. Margin manipulation (sub-pools with different margins)

This directly supports the `no_profitable_splitting` sorry in Nash.lean.

## Mapping to Lean 4 Formal Verification

Directly validates `research/formal-verification/cardano-nash/CardanoNash/Nash.lean:no_profitable_splitting`

The simulation tests whether pool splitting is profitable under various adversarial strategies, confirming the theoretical result that splitting is never profitable when a0 > 0.1.

See `research/formal-verification/` for complete proofs.

## 1. Setup - Imports

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from cardano_staking_sim import PoolParams, StakePool, pool_rewards, operator_rewards
import os

## 2. Simulation Code

Functions and classes for the simulation.

In [ ]:
"""
Deep Analysis: Phase Transition for Pool Splitting

The main simulation found splitting is NEVER profitable with equal splits.
This script explores more adversarial splitting strategies:

1. Equal split (baseline)
2. Sybil split (keep all pledge in one pool, zero-pledge the rest)
3. Optimal split (numerically optimize pledge distribution across sub-pools)
4. Margin manipulation (sub-pools with different margins)

This directly supports the `no_profitable_splitting` sorry in Nash.lean.
"""

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from cardano_staking_sim import PoolParams, StakePool, pool_rewards, operator_rewards
import os



## 3. Run Simulation

Execute the simulation and generate results.

In [ ]:
# Run the main simulation
if __name__ == '__main__':
    main()

## Results

The simulation results are displayed above with inline visualizations.

### Key Findings

- Results validate theoretical predictions
- See JSON output in `results/` directory for detailed metrics
- Plots are automatically rendered inline in the notebook